<a href="https://colab.research.google.com/github/Abhi-213/Community-detection/blob/main/CA_HepPh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
from community import community_louvain

# Load the directed graph from the edge list
def load_directed_graph_from_edgelist(filename):
    # Create a directed graph
    graph = nx.read_edgelist(filename, delimiter="\t", create_using=nx.DiGraph(), nodetype=int)
    return graph

# Calculate node importance
def calculate_imp(graph, alpha=0.7, beta=0.3, gamma=6):
    imp = {node: 1.0 for node in graph.nodes}  # Initialize importance
    for _ in range(gamma):  # Iterate to compute importance values
        new_imp = {}
        for node in graph.nodes:
            first_neighbors = list(graph.neighbors(node))
            second_neighbors = set()
            for neighbor in first_neighbors:
                second_neighbors.update(graph.neighbors(neighbor))

            imp_value = 0
            for neighbor in first_neighbors:
                if graph.has_edge(node, neighbor):
                    weight = graph[node][neighbor].get('weight', 1)
                    neighbor_weight_sum = sum(
                        graph[neighbor][n].get('weight', 1) for n in graph.neighbors(neighbor)
                        if graph.has_edge(neighbor, n)
                    )
                    if neighbor_weight_sum > 0:
                        imp_value += alpha * weight * imp[neighbor] / neighbor_weight_sum

            for second_neighbor in second_neighbors:
                if second_neighbor != node and not graph.has_edge(node, second_neighbor):
                    if graph.has_edge(node, second_neighbor):
                        weight = graph[node][second_neighbor].get('weight', 1)
                        second_neighbor_weight_sum = sum(
                            graph[second_neighbor][n].get('weight', 1) for n in graph.neighbors(second_neighbor)
                            if graph.has_edge(second_neighbor, n)
                        )
                        if second_neighbor_weight_sum > 0:
                            imp_value += beta * weight * imp[second_neighbor] / second_neighbor_weight_sum

            new_imp[node] = imp_value

        imp = new_imp
    return imp

# Form initial communities
def form_initial_communities(graph, imp):
    sorted_nodes = sorted(imp, key=imp.get, reverse=True)
    communities = []
    assigned = set()

    for node in sorted_nodes:
        if node not in assigned:
            community = {node}
            community.update(graph.neighbors(node))
            communities.append(community)
            assigned.update(community)

    return communities

# Calculate similarity between nodes using GLHN similarity
def glhn_similarity(graph, node1, node2):
    neighbors1 = set(graph.neighbors(node1))
    neighbors2 = set(graph.neighbors(node2))
    shared_neighbors = neighbors1.intersection(neighbors2)
    return len(shared_neighbors) / (len(neighbors1) * len(neighbors2)) ** 0.5

# Calculate the similarity of a node with a community
def calculate_similarity(graph, node, community):
    return sum(glhn_similarity(graph, node, member) for member in community)

# Resolve overlapping communities
def resolve_overlaps(graph, communities):
    final_communities = []
    for community in communities:
        overlaps = set()
        for node in community:
            for other_community in communities:
                if node in other_community and community != other_community:
                    overlaps.add(node)

        for overlap_node in overlaps:
            best_community = max(communities, key=lambda c: calculate_similarity(graph, overlap_node, c))
            if best_community != community:
                community.remove(overlap_node)

        final_communities.append(community)

    return final_communities

# Merge small or weak communities
def merge_small_weak_communities(graph, communities, mc=4):
    merged_communities = []
    for community in communities:
        internal_edges = sum(1 for node in community for neighbor in graph.neighbors(node) if neighbor in community)
        external_edges = sum(1 for node in community for neighbor in graph.neighbors(node) if neighbor not in community)

        if internal_edges <= mc * external_edges:
            best_community = max(communities, key=lambda c: calculate_similarity(graph, list(community)[0], c))
            merged_communities.append(best_community)
        else:
            merged_communities.append(community)

    return merged_communities

# Calculate modularity for a given partition
def calculate_modularity(graph, communities):
    partition = {}
    for idx, community in enumerate(communities):
        for node in community:
            partition[node] = idx

    # Ensure all nodes are in the partition
    for node in graph.nodes:
        if node not in partition:
            partition[node] = -1  # Assign to a default community

    modularity = community_louvain.modularity(partition, graph)
    return modularity

# LCD-SN Algorithm
def lcd_sn_algorithm(graph, alpha=0.7, beta=0.3, gamma=6, mc=4):
    imp = calculate_imp(graph, alpha, beta, gamma)
    communities = form_initial_communities(graph, imp)
    communities = resolve_overlaps(graph, communities)
    final_communities = merge_small_weak_communities(graph, communities, mc)
    modularity_value = calculate_modularity(graph, final_communities)
    return final_communities, modularity_value

# Example usage
if __name__ == '__main__':
    # Load the directed graph from the .txt file (for the Email-Enron dataset)
    G = load_directed_graph_from_edgelist('/content/CA-HepPh.txt')

    # Convert the directed graph to an undirected graph
    G_undirected = G.to_undirected()

    # Check if the undirected graph has any edges
    if G_undirected.number_of_edges() == 0:
        print("The graph has no edges.")
    else:
        print(f"The graph has {G_undirected.number_of_nodes()} nodes and {G_undirected.number_of_edges()} edges.")

    # Run LCD-SN Algorithm if the graph has edges
    if G_undirected.number_of_edges() > 0:
        final_communities, modularity_value = lcd_sn_algorithm(G_undirected)

        # Output final communities
        for idx, community in enumerate(final_communities):
            print(f"Community {idx + 1}: {sorted(community)}")

        # Output modularity value
        print(f"Modularity: {modularity_value}")


The graph has 12008 nodes and 118521 edges.
Community 1: [120, 131, 454, 595, 611, 662, 1077, 1106, 1234, 1403, 1610, 1851, 1911, 2932, 3070, 3257, 3362, 3737, 3885, 3919, 4023, 4450, 4725, 4966, 4982, 5101, 5109, 5363, 5394, 5574, 5732, 5790, 6090, 6182, 6429, 6526, 6777, 6782, 6852, 6879, 7181, 7369, 7562, 7613, 7770, 7860, 7913, 8114, 8168, 8265, 8371, 8480, 9057, 9238, 9290, 9381, 9681, 9799, 9935, 9936, 10306, 10326, 10358, 10450, 11187, 11188, 11279, 11334, 11363, 11466, 11544, 11879, 12011, 12087, 12416, 12500, 12665, 12712, 12728, 12735, 13097, 13198, 13313, 13323, 13377, 13495, 13781, 13797, 14222, 14274, 14431, 14527, 14558, 14662, 14989, 15419, 15644, 15986, 15997, 16038, 16320, 16374, 16667, 17051, 17120, 17132, 17201, 17202, 17284, 17337, 17376, 17440, 17583, 17666, 17912, 18068, 18110, 18198, 18262, 18291, 18378, 18418, 18976, 19085, 19236, 19918, 20087, 20117, 20217, 20269, 20378, 20421, 20445, 20474, 21331, 21566, 21634, 21824, 22061, 22208, 22394, 22506, 22664, 22846, 

In [5]:
import networkx as nx
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
import matplotlib.pyplot as plt
from scipy.io import mmread

# Function to calculate the importance of nodes
def calculate_imp(graph, alpha=0.7, beta=0.3, max_iter=6):
    importance = {node: 1.0 for node in graph.nodes()}
    for _ in range(max_iter):
        new_importance = {}
        for node in graph.nodes():
            neighbors = list(graph.neighbors(node))
            imp_score = 0
            for neighbor in neighbors:
                neighbor_imp = importance[neighbor]
                neighbor_degree = len(list(graph.neighbors(neighbor)))
                imp_score += alpha * (neighbor_imp / neighbor_degree)
            new_importance[node] = imp_score
        importance = new_importance
    return importance

# First Phase: Formation of Initial Communities
def form_initial_communities(graph, importance):
    communities = []
    sorted_nodes = sorted(importance, key=importance.get, reverse=True)
    visited = set()

    for node in sorted_nodes:
        if node not in visited:
            community = set([node]) | set(graph.neighbors(node))
            communities.append(community)
            visited.update(community)

    print(f"Initial communities formed: {communities}")
    return communities

# Calculate similarity of node to a community
def calculate_similarity(node, community, graph):
    neighbors = set(graph.neighbors(node))
    community_nodes = community
    common_neighbors = neighbors & community_nodes
    if len(neighbors) == 0:
        return 0
    return len(common_neighbors) / len(neighbors)  # Example similarity measure

# Second Phase: Determining Status of Overlapping Nodes
def assign_overlapping_nodes(graph, communities):
    node_to_communities = {}
    for community in communities:
        for node in community:
            if node in node_to_communities:
                node_to_communities[node].append(community)
            else:
                node_to_communities[node] = [community]

    final_communities = []
    assigned_nodes = set()

    for node, comms in node_to_communities.items():
        if node not in assigned_nodes:
            best_community = max(comms, key=lambda c: calculate_similarity(node, c, graph))
            for community in comms:
                if community != best_community:
                    community.discard(node)
            if best_community not in final_communities:
                final_communities.append(best_community)
            assigned_nodes.add(node)

    # Ensure all nodes are covered
    all_nodes = set(graph.nodes())
    covered_nodes = set().union(*final_communities)
    missing_nodes = all_nodes - covered_nodes
    if missing_nodes:
        for node in missing_nodes:
            final_communities.append({node})

    print(f"Communities after resolving overlaps: {final_communities}")
    return final_communities

# Third Phase: Integration of Communities
def merge_communities(graph, communities, mc=1.5):
    # Merge small communities
    small_communities = [c for c in communities if len(c) < 3]
    for community in small_communities:
        best_community = max(communities, key=lambda c: calculate_similarity(next(iter(community)), c, graph))
        if len(best_community) + len(community) < len(graph.nodes()):
            best_community.update(community)

    # Merge weak communities
    final_communities = []
    for community in communities:
        E_in = sum(1 for u in community for v in community if graph.has_edge(u, v))
        E_out = sum(1 for u in community for v in graph.nodes() if v not in community and graph.has_edge(u, v))
        if E_in <= mc * E_out:
            best_community = max(communities, key=lambda c: calculate_similarity(next(iter(community)), c, graph))
            if len(best_community) + len(community) < len(graph.nodes()):
                best_community.update(community)
        else:
            final_communities.append(community)

    # Ensure all nodes are covered after merging
    all_nodes = set(graph.nodes())
    covered_nodes = set().union(*final_communities)
    missing_nodes = all_nodes - covered_nodes
    if missing_nodes:
        for node in missing_nodes:
            final_communities.append({node})

    print(f"Communities after merging: {final_communities}")
    return final_communities

# Main function to run the LCD-SN algorithm
def lcd_sn_algorithm(graph, alpha=0.7, beta=0.3, max_iter=6, mc=1.5):
    importance = calculate_imp(graph, alpha, beta, max_iter)
    print(f"Node importance: {importance}")

    initial_communities = form_initial_communities(graph, importance)
    communities = assign_overlapping_nodes(graph, initial_communities)
    final_communities = merge_communities(graph, communities, mc)

    return final_communities

# Convert overlapping communities to a partition (disjoint communities)
def convert_to_partition(communities):
    node_to_community = {}
    for community in communities:
        for node in community:
            # Assign the node to the first community it appears in
            if node not in node_to_community:
                node_to_community[node] = community

    # Create disjoint communities from node assignments
    partition = {}
    for node, comm in node_to_community.items():
        comm_id = tuple(comm)
        if comm_id not in partition:
            partition[comm_id] = set()
        partition[comm_id].add(node)

    return list(partition.values())

# Calculate Modularity
def calculate_modularity(graph, communities):
    return nx.algorithms.community.quality.modularity(graph, communities)

# Visualize the final community graph
def visualize_communities(graph, communities):
    pos = nx.spring_layout(graph)
    colors = ['r', 'g', 'b', 'y', 'c', 'm']
    for i, community in enumerate(communities):
        nx.draw_networkx_nodes(graph, pos, nodelist=list(community), node_color=colors[i % len(colors)], label=f"Community {i+1}")
    nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_labels(graph, pos)
    plt.legend()
    plt.show()

# Load the graph from the .txt file
def load_graph_from_txt(file_path, directed=False):
    if directed:
        G = nx.read_edgelist(file_path, delimiter="\t", create_using=nx.DiGraph(), nodetype=int)
    else:
        G = nx.read_edgelist(file_path, delimiter="\t", create_using=nx.Graph(), nodetype=int)
    return G

# Validate the communities detected
def validate_communities(graph, communities):
    all_nodes = set(graph.nodes())
    community_nodes = set().union(*communities)

    if all_nodes != community_nodes:
        print("Warning: Not all nodes are covered in the detected communities.")
        print(f"Graph nodes: {all_nodes}")
        print(f"Covered nodes: {community_nodes}")
        print(f"Missing nodes: {all_nodes - community_nodes}")

    non_empty_communities = [community for community in communities if community]
    if not non_empty_communities:
        print("Error: No valid communities found.")
        return False

    return True

# Example Usage
if __name__ == "__main__":
    # Load the graph from the .txt file (directed or undirected)
    file_path = '/content/CA-HepPh.txt'  # Update this path if needed
    G = load_graph_from_txt(file_path, directed=True)  # Set to True if the graph is directed

    # If you want to work with an undirected version, you can convert the graph:
    G = G.to_undirected()

    # Run the LCD-SN algorithm
    communities = lcd_sn_algorithm(G)

    # Validate communities
    if validate_communities(G, communities):
        # Convert overlapping communities to disjoint communities
        disjoint_communities = convert_to_partition(communities)

        # Calculate modularity for disjoint communities
        mod = calculate_modularity(G, disjoint_communities)
        print(f"Modularity: {mod}")

        # Visualize the disjoint communities
    else:
        print("Community detection failed. Please check the input data and algorithm.")


Node importance: {17010: 0.2260932701547673, 1943: 0.105966347521351, 2489: 0.2556320940013649, 3426: 0.015928730357377095, 4049: 0.19690792607276275, 16961: 0.27847850517512207, 17897: 0.03500412167121741, 26904: 0.10354483556026468, 27801: 0.4920885373276803, 32403: 0.15925277551575273, 35262: 0.6011783930522079, 40848: 0.02123025467250638, 44114: 0.19990666475556898, 45648: 0.4660255837017853, 45654: 0.10187879663406343, 47702: 0.021626164861509145, 49335: 0.42583358142972877, 55416: 0.20384018373441623, 56358: 0.5995378874195768, 64003: 0.1520963203838675, 64714: 0.37280149780169575, 66411: 0.6535508673015893, 71844: 0.14204605844694596, 72504: 0.12750333602152197, 79185: 0.4543478309523626, 84491: 0.7102537753809536, 5874: 0.02714173280499453, 14789: 0.22888873210367988, 33245: 0.07191906045271046, 34338: 0.02714173280499453, 35425: 0.2698326777917952, 45957: 0.09134113992297686, 50498: 0.5006625799336396, 64959: 0.3820768939676527, 71391: 0.21211915890423297, 73805: 0.07725076622